In [1]:
import pennylane as qml
import numpy as np
from sklearn.svm import SVC

# Define number of qubits and feature map
num_qubits = 2  # Using two qubits for binary classification

dev = qml.device("default.qubit", wires=num_qubits)

def feature_map(x):
    """Encodes classical data into quantum states."""
    for i in range(num_qubits):
        qml.Hadamard(wires=i)
        qml.RZ(x[i], wires=i)
    qml.CNOT(wires=[0, 1])
    qml.RZ(x[0] * x[1], wires=1)

def quantum_kernel(x1, x2):
    """Computes the squared inner product of two quantum states."""
    @qml.qnode(dev)
    def kernel_circuit():
        feature_map(x1)
        qml.adjoint(feature_map)(x2)
        return qml.probs(wires=range(num_qubits))
    return kernel_circuit()[0]  # Extracting the probability of |00>

# Generate synthetic training data
np.random.seed(42)
X_train = np.random.rand(10, num_qubits) * np.pi  # Normalize input
Y_train = np.random.choice([-1, 1], size=10)  # Binary labels

# Compute the quantum kernel matrix
kernel_matrix = np.array([[quantum_kernel(x1, x2) for x2 in X_train] for x1 in X_train])

# Train a classical SVM using the quantum kernel
svm = SVC(kernel="precomputed")
svm.fit(kernel_matrix, Y_train)

# Testing phase
X_test = np.random.rand(5, num_qubits) * np.pi
kernel_test = np.array([[quantum_kernel(x1, x2) for x2 in X_train] for x1 in X_test])
Y_pred = svm.predict(kernel_test)

# Output results
print("Test Data:", X_test)
print("Predicted Labels:", Y_pred)


Test Data: [[1.94271694 1.20153978]
 [3.08891093 1.46637888]
 [2.70158246 2.13724917]
 [1.41528514 0.0416731 ]
 [2.96001411 1.76962213]]
Predicted Labels: [1 1 1 1 1]


In [1]:
import pennylane as qml
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define number of qubits and feature map
num_qubits = 2  # Using two qubits for binary classification

dev = qml.device("default.qubit", wires=num_qubits)

def feature_map(x):
    """Encodes classical data into quantum states."""
    for i in range(num_qubits):
        qml.Hadamard(wires=i)
        qml.RZ(x[i], wires=i)
    qml.CNOT(wires=[0, 1])
    qml.RZ(x[0] * x[1], wires=1)

def quantum_kernel(x1, x2):
    """Computes the squared inner product of two quantum states."""
    @qml.qnode(dev)
    def kernel_circuit():
        feature_map(x1)
        qml.adjoint(feature_map)(x2)
        return qml.probs(wires=range(num_qubits))
    return kernel_circuit()[0]  # Extracting the probability of |00>

# Load and preprocess Titanic dataset
df_train = pd.read_csv('train.csv')
df_train['Pclass'] = df_train['Pclass'].astype(str)
df_train = pd.concat([df_train, pd.get_dummies(df_train[['Pclass', 'Sex', 'Embarked']])], axis=1)
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())
df_train['is_child'] = df_train['Age'].map(lambda x: 1 if x < 12 else 0)
cols_model = ['is_child', 'Pclass_1', 'Pclass_2', 'Sex_female']

X_train, X_test, y_train, y_test = train_test_split(
    df_train[cols_model], df_train['Survived'], test_size=0.10, random_state=42, stratify=df_train['Survived']
)

X_train = np.array(X_train.values)
Y_train = np.array(y_train.values * 2 - np.ones(len(y_train)))
X_test = np.array(X_test.values)
Y_test = np.array(y_test.values * 2 - np.ones(len(y_test)))

# Compute the quantum kernel matrix
kernel_matrix = np.array([[quantum_kernel(x1, x2) for x2 in X_train] for x1 in X_train])

# Train a classical SVM using the quantum kernel
svm = SVC(kernel="precomputed")
svm.fit(kernel_matrix, Y_train)

# Testing phase
kernel_test = np.array([[quantum_kernel(x1, x2) for x2 in X_train] for x1 in X_test])
Y_pred = svm.predict(kernel_test)

# Evaluate model performance
print("Quantum Model Performance:")
print("Accuracy:", accuracy_score(Y_test, Y_pred))
print("Precision:", precision_score(Y_test, Y_pred))
print("Recall:", recall_score(Y_test, Y_pred))
print("F1 Score:", f1_score(Y_test, Y_pred, average='macro'))


Quantum Model Performance:
Accuracy: 0.6555555555555556
Precision: 0.5714285714285714
Recall: 0.45714285714285713
F1 Score: 0.6214896214896215


In [3]:
import pennylane as qml
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define number of qubits and feature map
num_qubits = 2  # Using two qubits for binary classification

dev = qml.device("default.qubit", wires=num_qubits)

def feature_map(x):
    """Encodes classical data into quantum states."""
    for i in range(num_qubits):
        qml.Hadamard(wires=i)
        qml.RZ(x[i], wires=i)
    qml.CNOT(wires=[0, 1])
    qml.RZ(x[0] * x[1], wires=1)

def quantum_kernel(x1, x2):
    """Computes the squared inner product of two quantum states."""
    @qml.qnode(dev)
    def kernel_circuit():
        feature_map(x1)
        qml.adjoint(feature_map)(x2)
        return qml.probs(wires=range(num_qubits))
    return kernel_circuit()[0]  # Extracting the probability of |00>

# Load and preprocess Titanic dataset
df_train = pd.read_csv('train.csv')
df_train['Pclass'] = df_train['Pclass'].astype(str)
df_train = pd.concat([df_train, pd.get_dummies(df_train[['Pclass', 'Sex', 'Embarked']])], axis=1)
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())
df_train['is_child'] = df_train['Age'].map(lambda x: 1 if x < 12 else 0)
cols_model = ['is_child', 'Pclass_1', 'Pclass_2', 'Sex_female']

X_train, X_test, y_train, y_test = train_test_split(
    df_train[cols_model], df_train['Survived'], test_size=0.10, random_state=42, stratify=df_train['Survived']
)

X_train = np.array(X_train.values)
Y_train = np.array(y_train.values * 2 - np.ones(len(y_train)))
X_test = np.array(X_test.values)
Y_test = np.array(y_test.values * 2 - np.ones(len(y_test)))

# Compute the quantum kernel matrix
kernel_matrix = np.array([[quantum_kernel(x1, x2) for x2 in X_train] for x1 in X_train])

# Train a classical SVM using the quantum kernel
svm_quantum = SVC(kernel="precomputed")
svm_quantum.fit(kernel_matrix, Y_train)

# Testing phase
kernel_test = np.array([[quantum_kernel(x1, x2) for x2 in X_train] for x1 in X_test])
Y_pred_quantum = svm_quantum.predict(kernel_test)

# Evaluate quantum model performance
print("Quantum Model Performance:")
print("Accuracy:", accuracy_score(Y_test, Y_pred_quantum))
print("Precision:", precision_score(Y_test, Y_pred_quantum))
print("Recall:", recall_score(Y_test, Y_pred_quantum))
print("F1 Score:", f1_score(Y_test, Y_pred_quantum, average='macro'))

# Train a classical SVM with RBF kernel
svm_classical = SVC(kernel="rbf", gamma='scale')
svm_classical.fit(X_train, Y_train)
Y_pred_classical = svm_classical.predict(X_test)

# Evaluate classical SVM performance
print("\nClassical SVM Performance:")
print("Accuracy:", accuracy_score(Y_test, Y_pred_classical))
print("Precision:", precision_score(Y_test, Y_pred_classical))
print("Recall:", recall_score(Y_test, Y_pred_classical))
print("F1 Score:", f1_score(Y_test, Y_pred_classical, average='macro'))


Quantum Model Performance:
Accuracy: 0.6555555555555556
Precision: 0.5714285714285714
Recall: 0.45714285714285713
F1 Score: 0.6214896214896215

Classical SVM Performance:
Accuracy: 0.7333333333333333
Precision: 0.7894736842105263
Recall: 0.42857142857142855
F1 Score: 0.6825396825396826
